In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.constants as con

# class holding initial conditions for arguments in
class Initial_Conditions:
    def __init__(init, arg1, arg2, arg3, arg4, arg5, arg6, arg7, arg8, arg9):
        """
        Args: in numeric order specific for Double Pendulum
            g: gravity
            l1: length of upper pendulum in meter
            l2: length of lower pendulum in meter
            m1: attached mass on the upper pendulum in kg
            m2: attached mass on the lower pendulum in kg
            theta1: initial angular space of the upper pendulum theta1, at t = 0
            theta2: initial angular space of the lower pendulum theta2, at t = 0
            omega1: initial angular velocity of the upper pendulum theta1, at t = 0
            omega2: initial angular velocity of the lower pendulum theta2, at t = 0
        Return:
            array of the above arguments
        """
        init.arg1 = arg1
        init.arg2 = arg2
        init.arg3 = arg3
        init.arg4 = arg4
        init.arg5 = arg5
        init.arg6 = arg6
        init.arg7 = arg7
        init.arg8 = arg8
        init.arg9 = arg9

#functions for calculating angular acceleration
def alpha_1(g, l1, l2, m1, m2, th1, th2, om1, om2):
    a1 = (-g*(2*m1 + m2)*np.sin(th1) - m2*g*np.sin(th1 - 2*th2) - 2*np.sin(th1 - th2)*m2*((om2**2)*l2 + (om1**2)*l1*np.cos(th1-th2)))/(l1*(2*m1 + m2 - m2*np.cos(2*th1 - 2*th2)))
    return a1

def alpha_2(g, l1, l2, m1, m2, th1, th2, om1, om2):
    a2 = (2*np.sin(th1 - th2)*((om1**2)*l1*(m1 + m2) + g*(m1 + m2)*np.cos(th1) + (om2**2)*l2*m2*np.cos(th1 - th2)))/(l2*(2*m1 + m2 - m2*np.cos(2*th1 - 2*th2)))
    return a2

#runge kutta step forward function for a 2nd order ODE
def rk4_step(con, kfunc1, kfunc2, step):
    """
    Args:
        con: array of arguments
        kfunc1: first k-function
        kfunc2: second k-fucntion
        step: time-step
    Return:
        theta1: array of zeros ready to be store values of theta1
        theta2: array of zeros ready to be store values of theta2
        omega1: array of zeros ready to be store values of omega1
        omega2: array of zeros ready to be store values of omega2
    """
    #calculating k1 for all values
    k1omega1 = step*kfunc1(con.arg1, con.arg2, con.arg3, con.arg4, con.arg5, con.arg6, con.arg7, con.arg8, con.arg9)
    k1omega2 = step*kfunc2(con.arg1, con.arg2, con.arg3, con.arg4, con.arg5, con.arg6, con.arg7, con.arg8, con.arg9)
    k1theta1 = step*con.arg8
    k1theta2 = step*con.arg9
    
    #calculating k2 for all values
    k2omega1 = step*kfunc1(con.arg1, con.arg2, con.arg3, con.arg4, con.arg5, con.arg6 + (k1theta1/2), con.arg7 + (k1theta2/2), con.arg8 + (k1omega1/2), con.arg9 + (k1omega2/2))
    k2omega2 = step*kfunc2(con.arg1, con.arg2, con.arg3, con.arg4, con.arg5, con.arg6 + (k1theta1/2), con.arg7 + (k1theta2/2), con.arg8 + (k1omega1/2), con.arg9 + (k1omega2/2))
    k2theta1 = step*(con.arg8 + (k1omega1/2))
    k2theta2 = step*(con.arg8 + (k1omega2/2))
    
    #calculating k3 for all values
    k3omega1 = step*kfunc1(con.arg1, con.arg2, con.arg3, con.arg4, con.arg5, con.arg6 + (k2theta1/2), con.arg7 + (k2theta2/2), con.arg8 + (k2omega1/2), con.arg9 + (k2omega2/2))
    k3omega2 = step*kfunc2(con.arg1, con.arg2, con.arg3, con.arg4, con.arg5, con.arg6 + (k2theta1/2), con.arg7 + (k2theta2/2), con.arg8 + (k2omega1/2), con.arg9 + (k2omega2/2))
    k3theta1 = step*(con.arg8 + (k2omega1/2))
    k3theta2 = step*(con.arg8 + (k2omega2/2))
    
    #calculating k4 for all values
    k4omega1 = step*kfunc1(con.arg1, con.arg2, con.arg3, con.arg4, con.arg5, con.arg6 + k3theta1, con.arg7 + k3theta2, con.arg8 + k3omega1, con.arg9 + k3omega2)
    k4omega2 = step*kfunc2(con.arg1, con.arg2, con.arg3, con.arg4, con.arg5, con.arg6 + k3theta1, con.arg7 + k3theta2, con.arg8 + k3omega1, con.arg9 + k3omega2)
    k4theta1 = step*(con.arg8 + k3omega1)
    k4theta2 = step*(con.arg8 + k3omega2)
    
    # runge-kutta solutions for double pendulum 
    theta1 = con.arg6 + (k1theta1/6) + (k2theta1/3) + (k3theta1/3) + (k4theta1/6)
    theta2 = con.arg7 + (k1theta2/6) + (k2theta2/3) + (k3theta2/3) + (k4theta2/6)
    omega1 = con.arg8 + (k1omega1/6) + (k2omega1/3) + (k3omega1/3) + (k4omega1/6)
    omega2 = con.arg9 + (k1omega2/6) + (k2omega2/3) + (k3omega2/3) + (k4omega2/6)
                
    return(theta1, theta2, omega1, omega2)

def DP_Solver(con, t, step):
    """
    Args:
        con: array of arguments from class
        t: end time for the simulation to run up to
        step: time-step to be step forward each time
    Return:
        t: array of time using time-step "step"
        theta1: initialised arrays with initial conditions ready to store in values of theta1
        theta2: initialised arrays with initial conditions ready to store in values of theta2
        omega1: initialised arrays with initial conditions ready to store in values of omega1
        omega2: initialised arrays with initial conditions ready to store in values of omega2
    """
    # Initialise the arrays to be used
    # t is an array containing each of the timepoints that we will step forward h to
    t = np.arange(0,t+(step),step)
    # n is the number of timesteps in t
    n = np.shape(t)[0]
    
    #initialising arrays which will store results
    theta1 = np.zeros(n)
    theta2 = np.zeros(n)
    omega1 = np.zeros(n)
    omega2 = np.zeros(n)
    
    #set initial conditions
    theta1[0] = con.arg6
    theta2[0] = con.arg7
    omega1[0] = con.arg8
    omega2[0] = con.arg9
    
    #for loop repeatedly steps forward in time using the 4th order runge kutta method
    for i in range(1,n):
        #class sets conditions for upcoming step
        c = Initial_Conditions(con.arg1, con.arg2, con.arg3, con.arg4, con.arg5, theta1[i-1], theta2[i-1], omega1[i-1], omega2[i-1])
        values = rk4_step(c, alpha_1, alpha_2, step)
        theta1[i] = values[0]
        theta2[i] = values[1]
        omega1[i] = values[2]
        omega2[i] = values[3]
    
    # keep the theta values between -pi to pi
    for i in range(len(theta1)):
        while theta1[i] > np.pi:
            theta1[i] = theta1[i] - 2*np.pi 
        while theta1[i] < -np.pi:
            theta1[i] = theta1[i] + 2*np.pi
    
    for j in range(len(theta2)):         
        while theta2[j] > np.pi:
            theta2[j] = theta2[j] - 2*np.pi 
        while theta2[j] < -np.pi:
            theta2[j] = theta2[j] + 2*np.pi

    return(t, theta1, theta2, omega1, omega2)
    

In [ ]:
# dummy reference to show choatic behaviour
c1 = Initial_Conditions(con.g, 1, 1, 1, 1, np.pi/2, np.pi/2, 0, 0)

x = DP_Solver(c1, 10, 0.01)

print(x[1], x[2])

[ 1.57079633  1.57030599  1.568835   ... -0.40568033 -0.39404573
 -0.38221683] [ 1.57079633  1.57079633  1.5699791  ... -0.53056294 -0.51719706
 -0.5036136 ]


In [ ]:
# change in theta1 (pi/2 to pi/2 + 0.1)
c2 = Initial_Conditions(con.g, 1, 1, 1, 1, np.pi/2 + 0.1, np.pi/2, 0, 0)

x = DP_Solver(c2, 10, 0.01)

print(x[1], x[2])

[ 1.67079633  1.67031324  1.66886376 ... -0.48512916 -0.48319909
 -0.4807392 ] [ 1.57079633  1.57078668  1.56996894 ... -0.44772529 -0.44158967
 -0.43503769]


In [ ]:
# change in theta2 (pi/2 to pi/2 + 0.1)
c3 = Initial_Conditions(con.g, 1, 1, 1, 1, np.pi/2, np.pi/2 + 0.1, 0, 0)

x = DP_Solver(c3, 10, 0.01)

print(x[1], x[2])

[ 1.57079633  1.57030601  1.56883519 ... -0.4776705  -0.46427834
 -0.45073288] [ 1.67079633  1.6707963   1.66997884 ... -0.699697   -0.68538507
 -0.67085087]


In [ ]:
# change in omega1 (0 to 1)
c4 = Initial_Conditions(con.g, 1, 1, 1, 1, np.pi/2, np.pi/2, 1, 0)

x = DP_Solver(c4, 10, 0.001)

print(x[1], x[2])

[ 1.57079633  1.57179142  1.57277671 ... -0.62622638 -0.62543899
 -0.62464865] [ 1.57079633  1.57162966  1.57245482 ... -0.79965366 -0.79851754
 -0.79737826]


In [ ]:
# change in omega2 (0 to 1)
c5 = Initial_Conditions(con.g, 1, 1, 1, 1, np.pi/2, np.pi/2, 0, 1)

x = DP_Solver(c5, 10, 0.001)

print(x[1], x[2])

[ 1.57079633  1.57079142  1.57077671 ... -0.76351137 -0.760036
 -0.7565557 ] [ 1.57079633  1.57096299  1.57112149 ... -0.85519076 -0.85213413
 -0.84907278]


In [ ]:
# theta outside desired range test
c6 = Initial_Conditions(con.g, 1, 1, 1, 1, np.pi, np.pi, 100, 100)

x = DP_Solver(c6, 10, 0.001)

print(x[1], x[2])

[ 3.14159265 -3.04159249 -2.94159135 ... -1.86306247 -1.7629436
 -1.66280982] [ 3.14159265 -3.04159265 -2.94159224 ... -1.86273292 -1.76266173
 -1.66257846]
